https://www.tensorflow.org/lite/examples/pose_estimation/overview?hl=ja

https://blog.tensorflow.org/2021/08/pose-estimation-and-classification-on-edge-devices-with-MoveNet-and-TensorFlow-Lite.html?hl=ja&_gl=1*xgz6jk*_ga*MTYwMDI2MzYzMC4xNzA0MTcwODE4*_ga_W0YLR4190T*MTcwNjUxOTU5NC41LjEuMTcwNjUyMDE0OC4wLjAuMA..

TFlite MOvenet RasPi sample code

https://github.com/tensorflow/examples/tree/master/lite/examples/pose_estimation/raspberry_pi#run-the-pose-classification-sample

Install dependencies. 
Run this script to install the Python dependencies, and download the TFLite models. sh setup.sh

for the error following:
ERROR: Could not find a version that satisfies the requirement tflite-runtime>=2.7.0 (from versions: none)
https://temcee.hatenablog.com/entry/tensorflow_install_error

In [ ]:
! sh setup.sh

In [ ]:
python3 pose_estimation.py

In [ ]:
! python3 pose_estimation.py

上記部分はmodule not found errorがでていたがカーネル再起動で通った

In [1]:
"""Main script to run pose classification and pose estimation."""
import argparse
import logging
import sys
import time

import cv2

import utils
from ml import Classifier
from ml import Movenet
from ml import MoveNetMultiPose
from ml import Posenet

In [2]:
def myRun(camera_id: int, width: int, height: int) -> None:
    model = 'movenet_multipose'
    pose_detector = MoveNetMultiPose(model,'bounding_box',512)

    # Variables to calculate FPS
    counter, fps = 0, 0
    start_time = time.time()

    # Start capturing video input from the camera
    cap = cv2.VideoCapture(camera_id)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)

    # Visualization parameters
    row_size = 20  # pixels
    left_margin = 24  # pixels
    text_color = (0, 0, 255)  # red
    font_size = 1
    font_thickness = 1
    classification_results_to_show = 3
    fps_avg_frame_count = 10
    keypoint_detection_threshold_for_classifier = 0.1
    classifier = None
    
    #key = cv2.waitKey(0)
    #cv2.startWindowThread()

    # Continuously capture images from the camera and run inference
    while cap.isOpened():
        success, image = cap.read()
        if not success:
          sys.exit(
              'ERROR: Unable to read from webcam. Please verify your webcam settings.'
          )

        counter += 1
        image = cv2.flip(image, 1)
        persons = pose_detector.detect(image)

        # Draw keypoints and edges on input image
        image = utils.visualize(image, persons)

        # Calculate the FPS
        if counter % fps_avg_frame_count == 0:
          end_time = time.time()
          fps = fps_avg_frame_count / (end_time - start_time)
          start_time = time.time()

        # Show the FPS
        fps_text = 'FPS = ' + str(int(fps))
        text_location = (left_margin, row_size)
        cv2.putText(image, fps_text, text_location, cv2.FONT_HERSHEY_PLAIN,
                    font_size, text_color, font_thickness)

        # Stop the program if the q key is pressed.
        if cv2.waitKey(1) == ord('q'):
          break
        cv2.imshow(model, image)
        '''
        key = cv2.waitKey(0)

        if key == ord('q'):            #qを押した時の処理
            cv2.waitKey(1)
            cv2.destroyAllWindows()  
            cap.release()
            cv2.waitKey(1)
            '''
            
    cv2.destroyAllWindows()  
    cap.release()

In [3]:
myRun(0, 1280, 720)

SystemExit: ERROR: Unable to read from webcam. Please verify your webcam settings.

/Users/nori/.pyenv/versions/3.9.16/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


qを押すとクラッシュしないように閉じる版

In [2]:
import cv2
import numpy as np
import serial
import time

import argparse
import logging
import sys
import utils

from ml import Classifier
from ml import Movenet
from ml import MoveNetMultiPose
from ml import Posenet

model = 'movenet_multipose'
pose_detector = MoveNetMultiPose(model,'bounding_box',512)

width = 1024
height = 720
camera_id = 0

# Start capturing video input from the camera
cap = cv2.VideoCapture(camera_id)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)

image = np.zeros((720, 1280,3), np.uint8)
cv2.putText(image, "hello", (10, 30),
               cv2.FONT_HERSHEY_PLAIN, 1.5,
               (255, 255, 255), 1, cv2.LINE_AA)

cv2.startWindowThread()
cv2.imshow(model, image)

# Visualization parameters
row_size = 20  # pixels
left_margin = 24  # pixels
text_color = (0, 0, 255)  # red
font_size = 1
font_thickness = 1
classification_results_to_show = 3
fps_avg_frame_count = 10
keypoint_detection_threshold_for_classifier = 0.1
classifier = None

start_time = time.time()

while cap.isOpened():
    end_time = time.time()
    
    time_diff = end_time - start_time #sec
    fps = 1.0 / time_diff
        
    start_time = time.time()
    
    success, image = cap.read()
    if not success:
      sys.exit(
          'ERROR: Unable to read from webcam. Please verify your webcam settings.'
      )

    image = cv2.flip(image, 1)
    persons = pose_detector.detect(image)

    # Draw keypoints and edges on input image
    image = utils.visualize(image, persons)
    
    #canvas = np.zeros((720, 1280,3), np.uint8)

    # Show the FPS
    fps_text = 'FPS = ' + str(int(fps))
    text_location = (left_margin, row_size)
    cv2.putText(image, fps_text, text_location, cv2.FONT_HERSHEY_PLAIN,
                font_size, text_color, font_thickness)
    time.sleep(0.01)
    
    cv2.imshow(model, image)
    
    key = cv2.waitKey(1)

    if key == ord('q'):            #qを押した時の処理
        cv2.waitKey(1)
        cv2.destroyAllWindows()  
        cap.release()
        cv2.waitKey(1)
        break

課題：opencvで画面を出して、配列５m分を用意して表示する

In [22]:
import cv2
import numpy as np
import serial
import time
import logging
import sys
import utils

from typing import List, Tuple
from data import Person

from ml import Classifier
from ml import Movenet
from ml import MoveNetMultiPose
from ml import Posenet

class Body:
    
def process_bodies(
    list_persons: List[Person],
    keypoint_threshold: float = 0.8,
    instance_threshold: float = 0.8,
)-> List[Body]:
    

def draw_bodies(
    image: np.ndarray,
    list_persons: List[Person],
    keypoint_color: Tuple[int, ...] = None,
    keypoint_threshold: float = 0.8,
    instance_threshold: float = 0.8,
) -> np.ndarray:
    for person in list_persons:
        if person.score < instance_threshold:
            continue
        keypoints = person.keypoints
        bounding_box = person.bounding_box
        
        # Draw bounding_box with multipose
        if bounding_box is not None:
            start_point = (bounding_box.start_point[0],int(height/2))
            end_point = (bounding_box.end_point[0],int(height/2))
            cv2.rectangle(image, start_point, end_point, (255, 255, 255), 2)
            # Draw id text when tracker is enabled for MoveNet MultiPose model.
            # (id = None when using single pose model or when tracker is None)
            if person.id:
                id_text = 'id = ' + str(person.id)
                cv2.putText(image, id_text, start_point, cv2.FONT_HERSHEY_PLAIN, 1,(0, 0, 255), 1)
    return image

model = 'movenet_multipose'
pose_detector = MoveNetMultiPose(model,'bounding_box',512)

width = 1024
height = 720
camera_id = 0

led_length = 150;  #5m 150
leds = [0] * led_length

# Start capturing video input from the camera
cap = cv2.VideoCapture(camera_id)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)

image = np.zeros((720, 1280,3), np.uint8)
cv2.putText(image, "hello", (10, 30),
               cv2.FONT_HERSHEY_PLAIN, 1.5,
               (255, 255, 255), 1, cv2.LINE_AA)

cv2.startWindowThread()
cv2.imshow(model, image)

# Visualization parameters
row_size = 20  # pixels
left_margin = 24  # pixels
text_color = (0, 0, 255)  # red
font_size = 1
font_thickness = 1
classification_results_to_show = 3
fps_avg_frame_count = 10
keypoint_detection_threshold_for_classifier = 0.1
classifier = None

start_time = time.time()

while cap.isOpened():
    end_time = time.time()
    
    time_diff = end_time - start_time #sec
    fps = 1.0 / time_diff
        
    start_time = time.time()
    
    success, image = cap.read()
    if not success:
      sys.exit(
          'ERROR: Unable to read from webcam. Please verify your webcam settings.'
      )

    image = cv2.flip(image, 1)
    persons = pose_detector.detect(image)

    # Draw keypoints and edges on input image
    image = utils.visualize(image, persons,keypoint_threshold=0.2,instance_threshold=0.2)
    image = draw_bodies(image, persons,keypoint_threshold=0.2,instance_threshold=0.2)

    # Show the FPS
    fps_text = 'FPS = ' + str(int(fps))
    text_location = (left_margin, row_size)
    cv2.putText(image, fps_text, text_location, cv2.FONT_HERSHEY_PLAIN,
                font_size, text_color, font_thickness)
    
    
    cv2.imshow(model, image)
    time.sleep(0.01)
    key = cv2.waitKey(1)

    if key == ord('q'):            #qを押した時の処理
        cv2.waitKey(1)
        cv2.destroyAllWindows()  
        cap.release()
        cv2.waitKey(1)
        break
        

    

2024-02-08 08:37:20.372 python3.9[11972:472091] _TIPropertyValueIsValid called with 4 on nil context!
2024-02-08 08:37:20.372 python3.9[11972:472091] imkxpc_getApplicationProperty:reply: called with incorrect property value 4, bailing.
2024-02-08 08:37:20.372 python3.9[11972:472091] Text input context does not respond to _valueForTIProperty:
2024-02-08 08:37:20.494 python3.9[11972:472091] _TIPropertyValueIsValid called with 4 on nil context!
2024-02-08 08:37:20.494 python3.9[11972:472091] imkxpc_getApplicationProperty:reply: called with incorrect property value 4, bailing.
2024-02-08 08:37:20.494 python3.9[11972:472091] Text input context does not respond to _valueForTIProperty:


課題：Serialに繋いで、バイト配列を送る